# Exploring NIH Funding

The NIH, as part of its open government initiative, provides information about extramural funding. [NIH Reporter](https://projectreporter.nih.gov/) is a web interface to the database containing information about all funded grants. The NIH also provides downloads of the grant information in either CSV or XML formats at [NIH ExPORTER](https://exporter.nih.gov/).

We will look at the 2015 funding to explore a little more about Pandas and to think about how we would design a smaller database from this data.

## File Formats

The data are stored in a CSV file that is stored with a [zip file](https://en.wikipedia.org/wiki/Zip_(file_format)). A zip file is a compressed archive of files. 

* We will use Python's [urllib](https://docs.python.org/3/library/urllib.html) package to read the file over the web
* We will use Python's [io.BytesIO](https://docs.python.org/3/library/io.html?highlight=io.bytesio#io.BytesIO) and the [zipfile](https://docs.python.org/3/library/zipfile.html) module to open the zip archive and extract individual files from it.
* We will use [Pandas](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) to read the CSV file into a dataframe.



In [ ]:
from zipfile import ZipFile
import os
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO

#### I had to usse the 'latin-1' encoding to get the file to read properly

In [ ]:
url="https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015.zip"

with urlopen(url) as f0:
    with ZipFile(BytesIO(f0.read())) as myzip:
        nih_data = pd.read_csv(myzip.open(myzip.namelist()[0]), 
                               encoding='latin-1')
nih_data.head()

## What is the shape of ``nih_data``?

#### What does a particular row of our data look like?

#### What are the column names?

#### How might I explore what the different values for a column are (e.g. 'ACTIVITY')?
* **Hint** Think uniqueness

In [ ]:
nih_data["ACTIVITY"].unique()

In [ ]:
selected_cols= \
["PI_NAMEs",
 "ORG_NAME",
 "ORG_DEPT",
 "TOTAL_COST",
 "TOTAL_COST_SUB_PROJECT",
 "ACTIVITY",
"PROJECT_TITLE"]

#### What Grants are more than $5,000,000 total cost?
* What States did they go to?
* Who are the principle investigators?

In [ ]:
nih_data[???][['ORG_STATE',"PI_NAMEs","TOTAL_COST"]]

#### What grants of greater than $5,000,000 were awared to an institution in Utah?

### Filtering with Strings

Filtering based on some numeric values is pretty straightforward. What about strings?

Filtering with exact matches is pretty straightforward also

In [ ]:
nih_data[nih_data["PI_NAMEs"]=='PARKER, DENNIS L;'][selected_cols]

### What if I don't know the name exactly?

### Introducing [string matching](http://pandas.pydata.org/pandas-docs/stable/text.html#testing-for-strings-that-match-or-contain-a-pattern) in Pandas

#### I'm interested in finding grants to my former chair Mike Becich, but I can't remember his middle initial/name (or know if he uses one or the other).
#### Use the contains method

In [ ]:
nih_data[nih_data["PI_NAMEs"].str.contains('BECICH')][selected_cols]

#### Look up the funding for the following researchers

* Ron Kikinis
* Carol Sweeney
* Matt Samore

#### Notice any inconsistencies in the data?

In [ ]:
nih_data[nih_data["PI_NAMEs"].str.contains('JOHNSON, K')][selected_cols]

#### Who were the NIH funded PIs at the University of Pittsburgh

In [ ]:
pitt_data = nih_data[???][???].unique()
pitt_data.sort()
pitt_data

#### If I don't know the NIH name of a Utah institution, how could I find it?

In [ ]:
nih_data[nih_data['ORG_NAME'].str.contains('UTAH')]

### Our vectorized indexing chokes on missing values

In [ ]:
nih_data["ORG_NAME"][nih_data["ORG_NAME"].str.contains("UTAH")==True].unique()

### What about searching by state?

In [ ]:
nih_data[nih_data['ORG_STATE']=='UT']["ORG_NAME"].unique()

In [ ]:
nih_data[nih_data["ORG_NAME"]=="UNIVERSITY OF UTAH"]

#### What was the total funding per PI at the University of Utah?
#### How many grants did each PI at the University of Utah have?

* I couldn't think of how to do this directly in Pandas so I used a defaultdict

In [ ]:
from collections import defaultdict
import numpy as np
ugrants = defaultdict(list)
for col, row in nih_data[nih_data["ORG_NAME"]=='UNIVERSITY OF UTAH'].iterrows():
    ugrants[row["PI_NAMEs"]].append(float(row['TOTAL_COST']))

#### Who brought in the most money?

In [ ]:
pd.DataFrame.from_dict({k:np.sum(v) for k,v in ugrants.items()}, 
                       orient='index').dropna().sort(columns=[0])

#### Plot a histogram of total funding/investigator

In [ ]:
fig1, ax1 = plt.subplots(1)
pd.DataFrame.from_dict({k:np.sum(v) for k,v in ugrants.items()}, 
                       orient='index').hist(bins=20, ax=ax1)
ax1.set_title("Grant Funding")
ax1.set_ylabel("# PIs")
ax1.set_xlabel("Total funding $")
fig1.savefig("./utah_funding_total.png")

In [ ]:
fig2, ax2 = plt.subplots(1)
pd.DataFrame.from_dict({k:len(v) 
                        for k,v in ugrants.items()}, 
                       orient='index').hist(bins=20, ax=ax2)
ax2.set_title("Number of Grants by PI")
ax2.set_ylabel("Count")
ax2.set_xlabel("Number of Grants")
fig2.savefig("./utah_numgrants.png")

## Database Design

### Design a relational database that would contain the following information

* PI name(s)
* Organization name
* Organization department (e.g. Radiology)
* Total cost
* Type of grant/contract
* Project title

What would the tables be? Data types?